Using kernel `conda_pytorch_latest_p36`

In [1]:
import sys
sys.path.append('../../../')

In [2]:
from pathlib import Path
import os
import random

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import torch

In [4]:
from deep.constants import *

In [5]:
%load_ext autoreload
%autoreload 2

## Data

In [6]:
data = FRAMEWORKS_PATH

In [7]:
afexportable = pd.read_csv(data /'afexportable_of_af_of_projects_of_interest.csv')
all_afs = pd.read_csv(data /'all_afs.csv')

proj_interest = pd.read_csv(data / 'projects_of_interest.csv')
entr_proj_interest = pd.read_csv(data / 'entries_of_projects_of_interest.csv')
exp_proj_interest = pd.read_csv(data / 'exportdata_of_entries_of_projects_of_interest.csv')
wid_proj_interest = pd.read_csv(data / 'widgets_of_afs_of_interest.csv')

/Users/stefano/miniconda3/envs/deep-exp/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
train = pd.read_csv(FRAMEWORKS_PATH / 'data_v0.3_train.csv')
val = pd.read_csv(FRAMEWORKS_PATH / 'data_v0.3_val.csv')
test = pd.read_csv(FRAMEWORKS_PATH / 'data_v0.3_test.csv')

/Users/stefano/miniconda3/envs/deep-exp/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
columns_to_keep = ['excerpt'] + DIMENSION_CLASSES

In [10]:
def process_for_sector(df, sector):
    df = df[['excerpt', sector]]
        
    return df

In [11]:
train_df = process_for_sector(train, 'Humanitarian Conditions')
test_df = process_for_sector(test, 'Humanitarian Conditions')

In [12]:
train_df.sample(1000).to_pickle('train_df.pickle', protocol=4)
test_df.sample(1000).to_pickle('test_df.pickle', protocol=4)

## Sagemaker Prep

In [13]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

sess = sagemaker.Session()

role = 'AmazonSageMaker-ExecutionRole-20210519T102514'
print(
    role
)  # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = SAGEMAKER_BUCKET
prefix = "huggingface/first"  # Replace with the prefix under which you want to store the data if needed


AmazonSageMaker-ExecutionRole-20210519T102514


### Bucket upload

In [29]:
bucket_path = 'test1/data'
train_channel = bucket_path + "/train_df.pickle"
validation_channel = bucket_path + "/test_df.pickle"

sess.upload_data(path="train_df.pickle", bucket=SAGEMAKER_BUCKET, key_prefix=bucket_path)
sess.upload_data(path="test_df.pickle", bucket=SAGEMAKER_BUCKET, key_prefix=bucket_path)

s3_train_data = f"s3://{SAGEMAKER_BUCKET}/{train_channel}"
s3_validation_data = f"s3://{SAGEMAKER_BUCKET}/{validation_channel}"

s3_output_location = f"s3://{SAGEMAKER_BUCKET}/{bucket_path}/output"

In [30]:
instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

In [31]:
metric_definitions=[
    {'Name': 'loss', 'Regex': "'loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'learning_rate', 'Regex': "'learning_rate': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_loss', 'Regex': "'eval_loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_accuracy', 'Regex': "'eval_accuracy': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_f1', 'Regex': "'eval_f1': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_precision', 'Regex': "'eval_precision': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_recall', 'Regex': "'eval_recall': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_runtime', 'Regex': "'eval_runtime': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_samples_per_second', 'Regex': "'eval_samples_per_second': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'epoch', 'Regex': "'epoch': ([0-9]+(.|e\-)[0-9]+),?"}]

In [32]:
from sagemaker.pytorch import PyTorch

hyperparameters={'epochs': 3,
                 'train_batch_size': 32,
                 'model_name': 'distilbert-base-uncased'
                 }

estimator = PyTorch(
    entry_point='train.py',
    source_dir=str(SCRIPTS_TRAINING_PATH / 'stefano/pytorch_estimator_base'),
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    role=role,
    framework_version='1.8',
    py_version='py36',
    hyperparameters = hyperparameters,
    metric_definitions=metric_definitions
#     entry_point="mnist.py",
#     role=role,
#     framework_version="1.4.0",
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
#     hyperparameters={"epochs": 6, "backend": "gloo"},
)

In [33]:
fit_arguments = {
    'train': f's3://{SAGEMAKER_BUCKET}/{bucket_path}',
    'test': f's3://{SAGEMAKER_BUCKET}/{bucket_path}'
}

In [34]:
estimator.fit(fit_arguments)

2021-05-25 15:03:07 Starting - Starting the training job...
2021-05-25 15:03:31 Starting - Launching requested ML instancesProfilerReport-1621954985: InProgress
......
2021-05-25 15:04:31 Starting - Preparing the instances for training.........
2021-05-25 15:06:12 Downloading - Downloading input data...
2021-05-25 15:06:52 Training - Downloading the training image...........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-05-25 15:08:48,523 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-05-25 15:08:48,548 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-05-25 15:08:48,557 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-05-25 15:08:49,177 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt

2021-05-25 15:

2021-05-25 15:09:07,904 - __main__ - INFO -  loaded train_dataset length is: (1000, 2)
2021-05-25 15:09:07,904 - __main__ - INFO -  loaded test_dataset length is: (1000, 2)
2021-05-25 15:09:08,105 - filelock - INFO - Lock 140409775294952 acquired on /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
2021-05-25 15:09:08,137 - filelock - INFO - Lock 140409775294952 released on /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
2021-05-25 15:09:08,162 - filelock - INFO - Lock 140409775294952 acquired on /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
2021-05-25 15:09:08,200 - filelock - INFO - Lock 140409775294952 released on 

{'eval_loss': 0.6906810998916626, 'eval_accuracy': 0.546, 'eval_f1': 0.5531496062992125, 'eval_precision': 0.49822695035460995, 'eval_recall': 0.6216814159292036, 'eval_runtime': 4.67, 'eval_samples_per_second': 214.133, 'epoch': 1.0}
{'eval_loss': 0.6725682616233826, 'eval_accuracy': 0.633, 'eval_f1': 0.5551515151515151, 'eval_precision': 0.613941018766756, 'eval_recall': 0.5066371681415929, 'eval_runtime': 4.5331, 'eval_samples_per_second': 220.597, 'epoch': 2.0}
{'eval_loss': 0.6320711374282837, 'eval_accuracy': 0.673, 'eval_f1': 0.6245694603903559, 'eval_precision': 0.649164677804296, 'eval_recall': 0.6017699115044248, 'eval_runtime': 4.5608, 'eval_samples_per_second': 219.262, 'epoch': 3.0}
{'train_runtime': 65.4287, 'train_samples_per_second': 1.467, 'epoch': 3.0}
***** Eval results *****
#015Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]#015Downloading: 100%|██████████| 232k/232k [00:00<00:00, 43.8MB/s]
#015Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]#015Down


2021-05-25 15:10:35 Uploading - Uploading generated training model
2021-05-25 15:11:14 Completed - Training job completed
Training seconds: 321
Billable seconds: 321
